In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import duckdb
import polars as pl
import sys
sys.path.append('../') 
import upsert_utils

In [3]:
duckdb.sql("""
attach '../../../db.sqlite3';
use db;
""")

In [4]:
# duckdb.sql("""
# rollback;
# """)

In [5]:
duckdb.sql("""
begin transaction;
""")

In [6]:
all_columns = duckdb.sql("""
select * from read_csv('ProdBattery.csv');
""").pl()

In [7]:
ob_dc_outputs = duckdb.sql(r"""
select
    csv_row_id,
    columns('^DCOutput__(.+)') as '\1',
from all_columns
""")
ob_dc_outputs

┌────────────┬───────────────────────────┬────────────────────────────┐
│ csv_row_id │ PowerDCContinuousMax_Unit │ PowerDCContinuousMax_Value │
│   int64    │          varchar          │           double           │
├────────────┼───────────────────────────┼────────────────────────────┤
│          1 │ kW                        │                        8.2 │
│          2 │ kW                        │                       5.76 │
│          3 │ kW                        │                      10.24 │
│          4 │ kW                        │                      20.48 │
│          5 │ kW                        │                      30.72 │
│          6 │ kW                        │                      40.96 │
│          7 │ kW                        │                       51.2 │
│          8 │ kW                        │                      61.44 │
│          9 │ kW                        │                      71.68 │
│         10 │ kW                        │                      

In [8]:
ob_prod_batteries = all_columns.drop('^.+__.+$')
duckdb.sql("""
describe ob_prod_batteries;
""")

┌─────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type │  null   │   key   │ default │  extra  │
│           varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ csv_row_id                  │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdMfr_Value               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdCode_Value              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Description_Value           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ BatteryChemistryType_Value  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ EnergyCapacityNominal_Unit  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ EnergyCapacityNominal_Value │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────────────────────┴─────────────┴─────────

In [9]:
ob_prod_battery_ids = upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select
        csv_row_id,
        BatteryChemistryType_Value,
        EnergyCapacityNominal_Unit,
        EnergyCapacityNominal_Value,
    from ob_prod_batteries
    """),
    target_table_name='server_prodbattery',
    pk='product_ptr_id',
    fk_fills=[
        ('product_ptr_id', dict(
            rows=duckdb.sql("""
            select
                csv_row_id,
                replace(cast(uuidv4() as varchar), '-', '') as ProdID_Value,
                ProdMfr_Value,
                ProdCode_Value,
                coalesce(Description_Value, '') as Description_Value,
                'ProdBattery' as ProdType_Value,
            from ob_prod_batteries
            """),
            target_table_name='server_product',
            pk='id',
            superclass=True,
            fk_fills=[
                ('Dimension_id', dict(target_table_name='server_dimension', pk='id')),
            ]
        )),
        ('DCInput_id', dict(target_table_name='server_dcinput', pk='id')),
        ('DCOutput_id', dict(rows=ob_dc_outputs, target_table_name='server_dcoutput', pk='id')),
    ]
)
ob_prod_battery_ids

┌────────────┬────────────────┐
│ csv_row_id │ product_ptr_id │
│   int64    │     int64      │
├────────────┼────────────────┤
│          1 │              1 │
│          2 │              2 │
│          3 │              3 │
│          4 │              4 │
│          5 │              5 │
│          6 │              6 │
│          7 │              7 │
│          8 │              8 │
│          9 │              9 │
│         10 │             10 │
│          · │              · │
│          · │              · │
│          · │              · │
│        854 │            854 │
│        855 │            855 │
│        856 │            856 │
│        857 │            857 │
│        858 │            858 │
│        859 │            859 │
│        860 │            860 │
│        861 │            861 │
│        862 │            862 │
│        863 │            863 │
├────────────┴────────────────┤
│ 863 rows          2 columns │
│ (20 shown)                  │
└─────────────────────────────┘

In [10]:
all_columns = duckdb.sql("""
select * from read_csv('ProdCertification.csv')
""").pl()

In [11]:
ob_certification_agencies = duckdb.sql(r"""
select
    row_number() over () as csv_row_id,
    replace(cast(uuidv4() as varchar), '-', '') as CertificationAgencyID_Value,
    *,
from (
    select distinct
        columns('^CertificationAgency__(.+)') as '\1',
    from all_columns
    where CertificationAgencyName_Value not in (select CertificationAgencyName_Value from server_certificationagency)
)
""")
ob_certification_agencies

┌────────────┬──────────────────────────────────┬────────────────────────────────┐
│ csv_row_id │   CertificationAgencyID_Value    │ CertificationAgencyName_Value  │
│   int64    │             varchar              │            varchar             │
├────────────┼──────────────────────────────────┼────────────────────────────────┤
│          1 │ e4434774b97047b3822a91ed4717cb33 │ SGS                            │
│          2 │ 4c140457e8fc463b8edd0861adcce05b │ TUV SUD America                │
│          3 │ aff1ca8623e74fe0b681ac710cfa8ddf │ CSA Group                      │
│          4 │ 5b9f0a3c05f5458aaeca3f32df99aa7e │ UL                             │
│          5 │ 9a9d93eb76234c808378da96f150162c │ TUV SUD                        │
│          6 │ d5943b14f51b435eafeaaecad80f53b1 │ TUV                            │
│          7 │ 4f4c945cb4a8406b9614de95b94ee904 │ TUV Rheinland                  │
│          8 │ 5e9bf3bb18234e8a8d8d8e718562b656 │ TUV Rheinland of North America │
│   

In [12]:
upsert_utils.insert_recursive(
    rows=ob_certification_agencies,
    target_table_name='server_certificationagency',
    pk='entity_ptr_id',
    fk_fills=[
        ('entity_ptr_id', dict(target_table_name='server_entity', pk='id', superclass=True)),
    ],
)

┌────────────┬───────────────┐
│ csv_row_id │ entity_ptr_id │
│   int64    │     int64     │
├────────────┼───────────────┤
│          1 │             1 │
│          2 │             2 │
│          3 │             3 │
│          4 │             4 │
│          5 │             5 │
│          6 │             6 │
│          7 │             7 │
│          8 │             8 │
│          9 │             9 │
└────────────┴───────────────┘

In [13]:
ob_prod_certifications = duckdb.sql("""
select
    ob_prod_battery_ids.product_ptr_id,
    all_columns.* exclude(CertificationAgency__CertificationAgencyName_Value, prodbattery_id),
    server_certificationagency.entity_ptr_id as CertificationAgency_id,
from all_columns
join server_certificationagency on CertificationAgencyName_Value = CertificationAgency__CertificationAgencyName_Value
join ob_prod_battery_ids on prodbattery_id = ob_prod_battery_ids.csv_row_id
""")
ob_prod_certifications

┌────────────────┬─────────────────────────┬─────────────────────────────┬────────────┬────────────────────────┐
│ product_ptr_id │ CertificationDate_Value │ CertificationStandard_Value │ csv_row_id │ CertificationAgency_id │
│     int64      │          date           │           varchar           │   int64    │         int64          │
├────────────────┼─────────────────────────┼─────────────────────────────┼────────────┼────────────────────────┤
│            863 │ 2024-11-27              │ UL1973_3_2022               │        863 │                      1 │
│            712 │ 2025-10-31              │ UL1973_3_2022               │        712 │                      2 │
│            849 │ 2022-12-27              │ UL1973_2_2018               │        849 │                      3 │
│            860 │ 2023-01-05              │ UL1973_3_2022               │        860 │                      4 │
│            801 │ 2023-01-04              │ UL1973_2_2018               │        801 │         

In [14]:
ob_prod_certification_ids = upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select * exclude(product_ptr_id) from ob_prod_certifications
    """),
    target_table_name='server_prodcertification',
    fk_fills=[
        ('Firmware_id', dict(
            target_table_name='server_firmware',
            pk='id',
            fk_fills=[
                ('Checksum_id', dict(target_table_name='server_checksum', pk='id')),
            ],
        )),
    ],
)
ob_prod_certification_ids

┌────────────┬───────┐
│ csv_row_id │  id   │
│   int64    │ int64 │
├────────────┼───────┤
│          1 │   863 │
│          2 │   394 │
│          3 │   749 │
│          4 │   747 │
│          5 │   745 │
│          6 │   743 │
│          7 │   741 │
│          8 │   739 │
│          9 │   737 │
│         10 │   735 │
│          · │     · │
│          · │     · │
│          · │     · │
│        854 │    16 │
│        855 │     8 │
│        856 │    25 │
│        857 │    29 │
│        858 │    21 │
│        859 │    13 │
│        860 │     4 │
│        861 │    17 │
│        862 │     9 │
│        863 │     1 │
├────────────┴───────┤
│      863 rows      │
│     (20 shown)     │
└────────────────────┘

In [15]:
upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select
        ob_prod_certification_ids.csv_row_id,
        product_ptr_id as product_id,
        ob_prod_certification_ids.id as prodcertification_id,
    from ob_prod_certification_ids
    join ob_prod_certifications using (csv_row_id)
    join ob_prod_battery_ids using (product_ptr_id)
    """),
    target_table_name='server_product_ProdCertifications',
)

┌────────────┬───────┐
│ csv_row_id │  id   │
│   int64    │ int64 │
├────────────┼───────┤
│        863 │     1 │
│        712 │     2 │
│        849 │     3 │
│        860 │     4 │
│        801 │     5 │
│        777 │     6 │
│        844 │     7 │
│        855 │     8 │
│        862 │     9 │
│        843 │    10 │
│          · │     · │
│          · │     · │
│          · │     · │
│         69 │   854 │
│         68 │   855 │
│         67 │   856 │
│         56 │   857 │
│         54 │   858 │
│         53 │   859 │
│         51 │   860 │
│         50 │   861 │
│         30 │   862 │
│          1 │   863 │
├────────────┴───────┤
│      863 rows      │
│     (20 shown)     │
└────────────────────┘

In [16]:
duckdb.sql("""
commit;
""")